<a href="https://colab.research.google.com/github/PatrickMarinich/Mario-Kart-Stats-Project/blob/main/Kartnite_Stats_Input.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KVR Helper Functions

In [61]:
import math
import random

#using the expectation of a win formula I developed, calculate the odds that racer 1 beats racer 2
#returns a value between 0 and 1
def getExpectedWinRate(kvr1,kvr2):
    pointDiff = abs(kvr1-kvr2)
    expterm = 0.5 * (1 - math.exp((-.0004 * pointDiff)))
    if kvr1 > kvr2:
        return 0.5+expterm, 0.5-expterm
    else:
        return 0.5-expterm, 0.5+expterm

    
#given an array of 12 kvrs, return the expected win array
def getEVArr(arr):
    EVs = []
    for i in range(0,len(arr)):
        playerScore = 0
        for j in range(0,len(arr)):
            if i != j:
                ev1,ev2 = getExpectedWinRate(arr[i],arr[j])
                playerScore = playerScore + ev1
        EVs.append(playerScore)

    return EVs
    
#Evarr is the expect amount of wins, however placement is the reverse of wins, so convert 
#ie. 12Wins -> first place or 2Wins -> tenth
def getPlacementExpected(arr):
    for i in range(0,len(arr)):
        arr[i] = 12 - arr[i]
    return arr

#given a place value, determine the amount of ingame points following the peicewise function below
def pointsExpected(placement):
    if(placement >=1 and placement <=2):
        points = (-3 * placement) + 18
    elif (placement > 2 and placement <= 4):
        points = (-2*placement) + 16
    else:
        points = (-1*placement) + 12
    return points


#gets the new KVR of the player given their current, expected points, and points scored
def getNewKVR(currKVR,ExPoints,actualPoints):
    return (currKVR + (20 * (actualPoints - ExPoints)))

#Input Functions
  Where all of the input data functions are located


In [62]:
import random

def inputRace(currentData,RaceCount,dfKartScore, dfPlacement,dfKVR, Track, Racers, Scores,TrackIndex):  
  #checks to see if the track exists
  if Track in set(currentData["Tracks x Players"]):
    print('Valid', Track)
    #seperates the players into an array, make sure that all players also exist

    racersArray = Racers.split()
    # checks that all players are valid
    validCount = 0
    for racer in racersArray:
      if(racer in set(currentData.columns.values.tolist())):
          print('Valid', racer)
          validCount = validCount + 1
      else: 
          validCount = validCount - 1
          print('Invalid', racer)

  
    if validCount == len(racersArray):
      print("....Everything is valid, Updating the scores....")

      ##if all players are valid update the scores

      #gets scores into an array
      if Scores == 'q':
          newScores = [15,12,10,8]
      else:
        newScores = Scores.split()
      #changes to integers
      for i in range(len(newScores)):
        newScores[i] = int(newScores[i])
      #Adds new Score to the current score for each racer, uses the track index to find the track
      for i in range(len(racersArray)):
        
        #only gives the points if there is a singular MVP
        if(len(getTrackOwner(currentData,RaceCount, Track, TrackIndex).split()) == 1):
        #gives the track owner a tally for each person that played the track
          dfKartScore.at[0, getTrackOwner(currentData,RaceCount, Track, TrackIndex)] = int(dfKartScore.at[0, getTrackOwner(currentData,RaceCount, Track, TrackIndex)]) + 1


        #if the score is then 15, 12, 10, or 8, then increment the placement stats
        FIRSTPLACE_ROW = 0
        TOP2_ROW = 1
        TOP3_ROW = 2
        TOP4_ROW = 3
        #adds to the row if that place or higher was achieved, this is for percentage stats later!
        if newScores[i] == 15:
          dfPlacement.at[FIRSTPLACE_ROW,racersArray[i]] =  int(dfPlacement.at[FIRSTPLACE_ROW,racersArray[i]]) + 1
          dfPlacement.at[TOP2_ROW,racersArray[i]] =  int(dfPlacement.at[TOP2_ROW,racersArray[i]]) + 1
          dfPlacement.at[TOP3_ROW,racersArray[i]] =  int(dfPlacement.at[TOP3_ROW,racersArray[i]]) + 1
          dfPlacement.at[TOP4_ROW,racersArray[i]] =  int(dfPlacement.at[TOP4_ROW,racersArray[i]]) + 1
        elif newScores[i] == 12:
          dfPlacement.at[TOP2_ROW,racersArray[i]] =  int(dfPlacement.at[TOP2_ROW,racersArray[i]]) + 1
          dfPlacement.at[TOP3_ROW,racersArray[i]] =  int(dfPlacement.at[TOP3_ROW,racersArray[i]]) + 1
          dfPlacement.at[TOP4_ROW,racersArray[i]] =  int(dfPlacement.at[TOP4_ROW,racersArray[i]]) + 1
        elif newScores[i] == 10:
          dfPlacement.at[TOP3_ROW,racersArray[i]] =  int(dfPlacement.at[TOP3_ROW,racersArray[i]]) + 1
          dfPlacement.at[TOP4_ROW,racersArray[i]] =  int(dfPlacement.at[TOP4_ROW,racersArray[i]]) + 1
        elif newScores[i] == 8:
          dfPlacement.at[TOP4_ROW,racersArray[i]] =  int(dfPlacement.at[TOP4_ROW,racersArray[i]]) + 1
        
        #increments the score
        currentData.at[TrackIndex[Track],racersArray[i]] =  int(currentData.at[TrackIndex[Track],racersArray[i]]) + newScores[i]
        #increments the count
        RaceCount.at[TrackIndex[Track],racersArray[i]] =  int(RaceCount.at[TrackIndex[Track],racersArray[i]]) + 1
        #---------------end of loop--------
      
      #where KVR elements are stored
      CURRKVR = 0

      #compute the new KVRs for the players
      player1KVR = round(float(dfKVR.at[CURRKVR,racersArray[0]]))
      player2KVR = round(float(dfKVR.at[CURRKVR,racersArray[1]]))
      player3KVR = round(float(dfKVR.at[CURRKVR,racersArray[2]]))
      KVRarray = [player1KVR,player2KVR,player3KVR]
      if len(racersArray) == 4:
        player4KVR = round(float(dfKVR.at[CURRKVR,racersArray[3]]))
        KVRarray.append(player4KVR)

      #get the random values for the coms
      while len(KVRarray) != 12:
        KVRarray.append(random.randint(4750,5250))

      print(KVRarray)

      #now the array is 12 racers long, compute the expected points for each of the players
      
      expectedPlacements = getPlacementExpected(getEVArr(KVRarray))
      expectedPoints = []
      for item in expectedPlacements: 
        expectedPoints.append(pointsExpected(item))

      print(expectedPoints)
      
      
      #expected points has all of the values for the players and cpus that they SHOULD get
      #index 0 is player 1, ... , index 3 is player 4

      #get their new KVRs
      player1New = getNewKVR(player1KVR,expectedPoints[0],newScores[0])
      player2New = getNewKVR(player2KVR,expectedPoints[1],newScores[1])
      player3New = getNewKVR(player3KVR,expectedPoints[2],newScores[2])
      newKVRs = [player1New,player2New,player3New]
      if len(racersArray) == 4:
        player4New = getNewKVR(player4KVR,expectedPoints[3],newScores[3])
        newKVRs.append(player4New)

      print(expectedPoints[0],newScores[0])
      print(expectedPoints[1],newScores[1])
      print(expectedPoints[2],newScores[2])
      print(expectedPoints[3],newScores[3])

      #update the data frame,
      HISTORYSTART = 2
      HISTORYEND = 51
      
      #outer loop is j inner loop is i, sorry i know it is backwards, but
      # it is what it is!
      for j in range(len(racersArray)):
        #load history 49->50
        #load history 48->49 
        #all the way until load 1->2
        for i in range (HISTORYEND,HISTORYSTART-1,-1):
          dfKVR.at[i,racersArray[j]] = round(float(dfKVR.at[i-1,racersArray[j]]))
        
        #load current into the first box in the history
        dfKVR.at[2,racersArray[j]] = dfKVR.at[0,racersArray[j]]

        #put the current KVR in the first box
        dfKVR.at[0,racersArray[j]] = round(newKVRs[j])

        #if this is a new all time max KVR, update it
        if newKVRs[j] > round(float(dfKVR.at[1,racersArray[j]])):
          dfKVR.at[1,racersArray[j]] = round(newKVRs[j])
  
  
        
    else: 
      print('There was an error entering the racers, please try again')
  else:
    print('There was an error entering the track name, please try again:', Track, "was invalid")





In [63]:
def editAScore(dfScores, Track, Racer, Score,TrackIndex):
 #makes sure track is valid
  if Track in set(dfScores["Tracks x Players"]):
    print('Valid', Track)
    # makes sure racer is valid
    if Racer in set(dfScores.columns.values.tolist()):
      print('Valid', Racer)
      #changes a specific score
      dfScores.at[TrackIndex[Track],Racer] =  int(dfScores.at[TrackIndex[Track],Racer]) + int(Score)
      print(Racer, '\'s Score was changed by ', Score, ' on ', Track)
    else:  
      print('There was an error entering the racer, please try again')
  else:
     print('There was an error entering the track name, please try again:', Track, "was invalid")



In [64]:
#allows for GP winners to be counted
def enterWinner(dfWins, Player):

  if Player in set(dfWins.columns.values.tolist()):
    
    dfWins.at[0, Player] = int(dfWins.at[0,Player]) + 1
    print(Player, 'has', dfWins.at[0,Player], 'wins')


  else: 
    print(Player, "does not exist within the Database, Please try again")



In [65]:
#allowes for Shock dodges
def enterDodges(dfShock, Player, Count):
  if Player in set(dfShock.columns.values.tolist()):
    
    dfShock.at[0, Player] = int(dfShock.at[0,Player]) + int(Count)
    print(Player, 'has', dfShock.at[0,Player], 'shock dodges')


  else: 
    print(Player, "does not exist within the Database, Please try again")


In [66]:
def addBlueShells(dfBlue, Player, hit, dodge):
  if Player in set(dfBlue.columns.values.tolist()):
    
    dfBlue.at[0, Player] = int(dfBlue.at[0,Player]) + int(hit)
    print(Player, 'has been hit by', dfBlue.at[0,Player], 'Blue Shells')
    dfBlue.at[1, Player] = int(dfBlue.at[1,Player]) + int(dodge)
    print(Player, 'has', dfBlue.at[1,Player], 'Blue Shell Dodges')
    
  else: 
    print(Player, "does not exist within the Database, Please try again")


# Stat Getter Functions

In [67]:
def GetTrackData(dfScores,dfRaceCount,Track,TrackIndex):
 
  AVERAGE_PERCENT = .96
  TOTAL_PERCENT = .04

  #eliminates the first row, to allow for players only
  dfNoTracks = dfScores[dfScores.columns.difference(["Tracks x Players"])]
 
  #make sure that the track exists in the data
  if Track in set(dfScores["Tracks x Players"]):
    print("Stats For: ", Track)
    #now display the data for the track plus any interesting stats
    print("\n", "Total Scores")
    print(dfScores.loc[[TrackIndex[Track]]])

    races = 0
    for player in set(dfNoTracks.columns.values.tolist()):
      races = races + int(dfRaceCount.at[TrackIndex[Track], player])
    print('\nTimes Played:',races )

    #prints player averages. 
    print('\nPlayer Averages:')
    for racer in set(dfNoTracks.columns.values.tolist()):
      print(racer,": ", getPlayerAverage(dfScores,dfRaceCount,racer,Track,TrackIndex))
    print('\n')
    print(getTrackOwner(dfScores, dfRaceCount, Track, TrackIndex), "is the MVP for this track")
    
    print('\nMVP Leaderboard')
    #determine the score for each player within the data
    dfNoTracks = dfScores[dfScores.columns.difference(["Tracks x Players"])]
    #leaderboard object
    dfLeaderboard = pd.DataFrame({'Player': [], 'MVP Points':[]})
    counter = 0
   
    #gets the ownership scores of each player for the track
    TrackTotalPoints = 0
    for player in set(dfNoTracks.columns.values.tolist()):
      TrackTotalPoints = TrackTotalPoints + int(dfScores.at[TrackIndex[Track], player])
    for player in set(dfNoTracks.columns.values.tolist()):
      ownershipScore = ((int(dfScores.at[TrackIndex[Track],player])/TrackTotalPoints)*100) *TOTAL_PERCENT + getPlayerAverage(dfScores,dfRaceCount,player,Track,TrackIndex) * AVERAGE_PERCENT


      #puts the score into the datafram
      dfLeaderboard.loc[counter] = [player, ownershipScore]
      counter = counter + 1
      playerScore = 0

    #sorts and prints the leaderboards
    dfLeaderboard = dfLeaderboard.sort_values(['MVP Points', 'Player'],  ascending=[0, 1])
    print(dfLeaderboard)

  else:
    print('Track Selection Was Invalid')


In [68]:
def getTrackOwner(dfScores,dfRaceCount, Track, TrackIndex):

  AVERAGE_PERCENT = .96
  TOTAL_PERCENT = .04

  #eliminates the first row, to allow for players only
  dfNoTracks = dfScores[dfScores.columns.difference(["Tracks x Players"])]


  #goes through all players and then find the max score

  currentMaxScore = 0
  currentPlayer = ""
  TrackTotalPoints = 0
  for player in set(dfNoTracks.columns.values.tolist()):
    TrackTotalPoints = TrackTotalPoints + int(dfScores.at[TrackIndex[Track], player])

  if TrackTotalPoints != 0:
    for player in set(dfNoTracks.columns.values.tolist()):
      #way to calculate track owner, total points + average
      playerScore = ((int(dfScores.at[TrackIndex[Track],player])/TrackTotalPoints)*100) *TOTAL_PERCENT + getPlayerAverage(dfScores,dfRaceCount,player,Track,TrackIndex) * AVERAGE_PERCENT
  
      if playerScore > currentMaxScore:
        currentPlayer = player
        currentMaxScore = playerScore
      elif playerScore == currentMaxScore:
        currentPlayer = currentPlayer + ", " + player
      else:
        continue

  return currentPlayer



In [69]:
def getPlayerAverage(dfScores, dfRaceCount, Player, Track, TrackIndex):
  #eliminates the first row, to allow for players only
  dfNoTracks = dfScores[dfScores.columns.difference(["Tracks x Players"])]
  
  if Player in set(dfNoTracks.columns.values.tolist()):
    #checks for divide by 0

    RaceCount = int(dfRaceCount.at[TrackIndex[Track],Player])
    Score =  int(dfScores.at[TrackIndex[Track],Player])
   
    if(Score == 0 or RaceCount == 0):
      averageScore = 0
    else: 
      averageScore = Score/RaceCount
  
    

  else:
    averageScore = "ERROR"
    print("There was an internal with Averaging Have Pat re-read code")
  

  return averageScore



# Season Reset Code

In [70]:
from pandas.core.dtypes.cast import sanitize_to_nanoseconds

#Point Values, constants which can be adjusted for ease of balancing
POINTS_SCORED_POINTS = 0.25        #points for each GP point scored
GP_WINS_POINTS = 125               #points per gp win
OWNER_POINTS = 0.25                #points per player on owned track
DODGE_POINTS = 3                 #points per dodge
AVERAGE_POINTS = 3                 #points for track average
BLUE_POINTS = 0.5                 #points for getting hit with blue shell
BLUE_D_POINTS = 8                  #points for dodging a blue shell


#seeds are needed to preform the reset
def getSeedings(sh,TrackIndex,display=True):
    
    kartData = sh.worksheet('Total Scores').get_all_values()
    RaceCount = sh.worksheet('Race Count').get_all_values()
    Wins = sh.worksheet('GP Wins').get_all_values()
    Shock = sh.worksheet('Shock Dodges').get_all_values()
    KartScore = sh.worksheet('Owned Score').get_all_values()
    Blue = sh.worksheet('Blue Shells').get_all_values()

    dfKartScore = pd.DataFrame(KartScore[1:], columns = KartScore[0])
    dfScores = pd.DataFrame(kartData[1:], columns=kartData[0])
    dfRaceCount = pd.DataFrame(RaceCount[1:], columns = RaceCount[0])
    dfWins = pd.DataFrame(Wins[1:], columns = Wins[0])
    dfShock = pd.DataFrame(Shock[1:], columns = Shock[0])
    dfBlue = pd.DataFrame(Blue[1:], columns = Blue[0])

    #determine the score for each player within the data
    dfNoTracks = dfScores[dfScores.columns.difference(["Tracks x Players"])]
    #leaderboard object
    dfLeaderboard = pd.DataFrame({'Player': [], 'Kart Score':[]})
    counter = 0
   
    #sets up the scores for each player
    for player in set(dfNoTracks.columns.values.tolist()):
      playerScore = 0
      #gp wins 
      playerScore = int(dfWins.at[0,player])*GP_WINS_POINTS
    
      #enters the owned track score 
      playerScore = playerScore + int(dfKartScore.at[0, player])*OWNER_POINTS


      #Dodges
      playerScore = int(dfShock.at[0,player])*DODGE_POINTS + playerScore
    
      #total points
      points = 0
      for track in TrackIndex:
        points = points + int(dfScores.at[TrackIndex[track], player]) * POINTS_SCORED_POINTS
      playerScore = playerScore + points

      #adds in track averages
      avgpoints = 0
      for track in TrackIndex:
        avgpoints = avgpoints + (int(getPlayerAverage(dfScores,dfRaceCount,player,track,TrackIndex))*AVERAGE_POINTS)
      playerScore = playerScore + avgpoints

      #adds blue points
      playerScore = playerScore + (int(dfBlue.at[0,player]) * BLUE_POINTS) + (int(dfBlue.at[1,player]) * BLUE_D_POINTS)

      #puts the score into the datafram
      dfLeaderboard.loc[counter] = [player, playerScore]
      counter = counter + 1
      playerScore = 0

    #sorts and prints the leaderboards
    dfLeaderboard = dfLeaderboard.sort_values(['Kart Score', 'Player'],  ascending=[0, 1])
    
    if display == True:
      print(dfLeaderboard)

    return dfLeaderboard





#this code takes in the name of the files, and then the sheet name, and then transfers all the data
def transfer_data(old,new,name):
  oldKart = old.worksheet(name).get_all_values()
  newKart= new.worksheet(name).get_all_values()
  dfOldKart = pd.DataFrame(oldKart[1:], columns = oldKart[0])
  dfNewKart = pd.DataFrame(newKart[1:], columns = newKart[0])

  #deleats first column
  colList = dfOldKart.columns.values.tolist();
  del colList[0]
 
  #iterates through the sheets
  for col in colList:
    for row in dfOldKart.index:
        dfNewKart.at[row,col] = float(dfNewKart.at[row,col]) + float(dfOldKart.at[row,col])
        #clears the old stats
        dfOldKart.at[row,col] = 0
 
  return dfNewKart,dfOldKart;


#This function will take all of the seasonal stats that have been aquired, 
#increment them into the all time sheet, and the reset the season for the next use
def end_season(season, all_time,TrackIndex):
  #season is the sheet for the current season
  #all time is the all time stats'

  #first get the seeding points and then update the kart sheet
  dfFinalRanks = getSeedings(season,TrackIndex)

  #find the score of each player using the decided algorythem, and then 
  #increment this score with the all time score
  
  totalScores = 0
  #get the total points,
  for index in dfFinalRanks.index:
    totalScores += dfFinalRanks.at[index,'Kart Score']

  
  #make a new leaderboard, and then add all the players to it with their new scores:
  dfSeasonScores = pd.DataFrame({'Player': [], 'Season Score':[]})
  for index in dfFinalRanks.index:
    #make score percent of score of the total:
    score = (int(dfFinalRanks.at[index,'Kart Score']) / totalScores) * 100
    #multiplier for placements
    if(index == 0):
      score *= 1.25
    elif(index == 1):
      score *= 1.125
    elif(index == 2):
      score *= 1.05

    dfSeasonScores.at[index] = [dfFinalRanks.at[index,'Player'],score]
    
  print("Below are the Final Scores Normalized for the Season")
  print(dfSeasonScores)


  #adds the new df to the all time scoring sheet
  all_timeScores= all_time.worksheet('All-Time Seeding').get_all_values()
  dfAll_timeScores = pd.DataFrame(all_timeScores[1:], columns = all_timeScores[0])  


  print('Updating All Time Seeding....')
  print('Diminishing Old Seasons by 15%...')
  print('Incrementing The New Seasons Scores...')
  
  for index in dfSeasonScores.index:
    player = dfSeasonScores.at[index,'Player']

  
    dfAll_timeScores.at[0,player] = (float(dfAll_timeScores.at[0,player])*.85) + float(dfSeasonScores.at[index,'Season Score'])

  #increments the all time stats with the seasonal stats, and then clears all of the seasonal stats

  #calls the transfer function on points score
  newPoints,oldPoints = transfer_data(season,all_time,'Total Scores')
  
  #calls it for the race count
  newRaceCount,oldRaceCount = transfer_data(season,all_time,'Race Count')

  #GP Wins
  newWins,oldWins = transfer_data(season,all_time,'GP Wins')

  #Shock Dodges
  newDodge,oldDodge = transfer_data(season,all_time,'Shock Dodges')

  #races owned
  newOwned,oldOwned = transfer_data(season,all_time,'Owned Score')

  #blue Shells
  newShells,oldShells = transfer_data(season,all_time,'Blue Shells')

  ##DO NOT TRANSFER KVR CODE


  return newPoints,newRaceCount,newWins,newDodge,newOwned,newShells,dfAll_timeScores,oldPoints,oldRaceCount,oldWins,oldDodge,oldOwned,oldShells

 


# I/O Logic

In [71]:
#This is the main bulk of the code, it culminates all of the previous functions 
#into the user inputted choices, this is the method that gets user inputs and does all 
#of the database scraping and saving for this program to work.
def RunKartniteStats():
### Patrick Marinich December 2021

  #constants
  VERSION_NUMBER = 'v3.3'
  CONTRIBUTORS = 'Patrick Marinich'


  #imports
  import pandas as pd
  from google.colab import auth
  auth.authenticate_user()
  import gspread
  from gspread_dataframe import get_as_dataframe, set_with_dataframe

  from google.auth import default
  creds, _ = default()

  gc = gspread.authorize(creds)

  # Puts the google sheet into a python object, we will be calling the sh object after a selection is made
  sh = gc.open_by_url('https://docs.google.com/spreadsheets/d/1G0z4XeBjG7Q_Zk_uPtV4v8BPkVT8GD9pmsuk7VIwFKc/edit#gid=1608384215')

  #Track Dictionary, all track names and nicknames go here
  #track indexes are starting at 0 in the mushroom cup ending at 31 at N64 Bowsers castle, +32 for Lava Lake
  TrackIndex = {"Luigi Circuit":0,
  "Moo Moo Meadows":1,
  "Mushroom Gorge":2,
  "Toad's Factory":3,
  "Mario Circuit":4,
  "Coconut Mall":5,
  "DK Summit":6,
  "Wario's Gold Mine":7,
  "Daisy Circuit":8,
  "Koopa Cape":9,
  "Maple Treeway":10,
  "Grumble Volcano":11,
  "Dry Dry Ruins":12,
  "Moonview Highway":13,
  "Bowser's Castle":14,
  "Rainbow Road":15,
  "GCN Peach Beach":16,
  "DS Yoshi Falls":17,
  "SNES Ghost Valley 2":18,
  "N64 Mario Raceway":19,
  "N64 Sherbet Land":20,
  "GBA Shy Guy Beach":21,
  "DS Delfino Square":22,
  "GCN Waluigi Stadium":23,
  "DS Desert Hills":24,
  "GBA Bowser's Castle 3":25,
  "N64 DK's Jungle Parkway":26,
  "GCN Mario Circuit":27,
  "SNES Mario Circuit 3":28,
  "DS Peach Gardens":29,
  "GCN DK Mountain":30,
  "N64 Bowser's Castle":31,
  "Lava Lake":32 }

#for nicknames 
  NickNameIndex= {"Luigi":"Luigi Circuit",
   "moo moo": "Moo Moo Meadows",
   "Moo Moo": "Moo Moo Meadows",
   "Gorge":"Mushroom Gorge",
   "gorge":"Mushroom Gorge",
   "toads":"Toad's Factory",
   "Toads":"Toad's Factory",
   "toads factory": "Toad's Factory",
   "mario circuit": "Mario Circuit",
   "coconut mall":"Coconut Mall",
   "coconut":"Coconut Mall",
   "summit":"DK Summit",
   "Summit":"DK Summit",
   "gold mine":"Wario's Gold Mine",
   "Gold Mine":"Wario's Gold Mine",
   "koopa": "Koopa Cape",
   "Koopa": "Koopa Cape",
   "maple":"Maple Treeway",
   "Maple":"Maple Treeway",
   "grumble":"Grumble Volcano",
   "Grumble":"Grumble Volcano",
   "Dry Dry":"Dry Dry Ruins",
   "dry dry":"Dry Dry Ruins",
   "Moonview":"Moonview Highway",
   "moonview":"Moonview Highway",
   "BC Wii":"Bowser's Castle",
   "bc wii":"Bowser's Castle",
   "BC wii":"Bowser's Castle",
   "rainbow road":"Rainbow Road",
   "rainbow":"Rainbow Road",
   "Rainbow":"Rainbow Road",
   "Peach Beach":"GCN Peach Beach",
   "peach beach":"GCN Peach Beach",
   "yoshi falls":"DS Yoshi Falls",
   "Yoshi Falls":"DS Yoshi Falls",
   "Ghost Valley":"SNES Ghost Valley 2",
   "ghost valley":"SNES Ghost Valley 2",
   "mario raceway":"N64 Mario Raceway",
   "raceway": "N64 Mario Raceway",
   "Raceway": "N64 Mario Raceway",
   "Sherbet Land":"N64 Sherbet Land",
   "sherbet land":"N64 Sherbet Land",
   "Shy Guy Beach":"GBA Shy Guy Beach",
   "shy guy beach":"GBA Shy Guy Beach",
   "Shy Guy":"GBA Shy Guy Beach",
   "shy guy":"GBA Shy Guy Beach",
   "Delfino Square":"DS Delfino Square",
   "delfino square":"DS Delfino Square",
   "Delfino":"DS Delfino Square",
   "delfino":"DS Delfino Square",
   "Waluigi Stadium":"GCN Waluigi Stadium",
   "waluigi stadium":"GCN Waluigi Stadium",
   "Waluigi":"GCN Waluigi Stadium", 
   "waluigi":"GCN Waluigi Stadium",
   "Desert Hills":"DS Desert Hills",
   "desert hills":"DS Desert Hills",
   "GBA 3":"GBA Bowser's Castle 3",
   "gba 3":"GBA Bowser's Castle 3",
   "BC3":"GBA Bowser's Castle 3",
   "bc3":"GBA Bowser's Castle 3",
   "Parkway":"N64 DK's Jungle Parkway",
   "parkway":"N64 DK's Jungle Parkway",
   "GCN mario circuit":"GCN Mario Circuit",
   "GCN mario":"GCN Mario Circuit",
   "gcn mario":"GCN Mario Circuit",
   "GCN Mario":"GCN Mario Circuit",
   "SNES 3":"SNES Mario Circuit 3",
   "snes 3":"SNES Mario Circuit 3",
   "Peach Gardens":"DS Peach Gardens",
   "peach gardens":"DS Peach Gardens",
   "DK Mountain": "GCN DK Mountain",
   "dk mountain": "GCN DK Mountain",
   "mountain": "GCN DK Mountain",
   "Mountain": "GCN DK Mountain",
   "BC64":"N64 Bowser's Castle",
   "bc64":"N64 Bowser's Castle",
   "bc 64":"N64 Bowser's Castle",
   "N64BC":"N64 Bowser's Castle",
   "n64bc":"N64 Bowser's Castle"}

  #Below is the logic for asking a user what they would like to do
  ########


  #displaying inputs for the user
  print('Welcome to Kartnite Stats ' ,VERSION_NUMBER , "\nDeveloped by:", CONTRIBUTORS, '\n')
  print('Below are the options, type the number of the option you would like')
  print('1. Input a Race \n2. Edit A Singular Score \n3. Enter A Grand Prix Winner \n4. Enter Shock Dodges \n5. Enter Blue Shell Stats')
 
  selection = input('What Would You Like To Do: ')
  
  #selection logic
  
  #depending on the user selection it will ask for different inputs, and load the nessassary databases needed for that operation

  #input a race
  if selection == '1':
    print('Warining: track and racer names must be formatted correctally \nTrack names must match their full name or a registered nickname \nRacer names must have their first letter capital')
    Track = input("Name of Track: ")
    if Track in NickNameIndex:
      Track = NickNameIndex[Track]
    Racers = input("Names of Racers: ")
    Scores = input("Scores:")
    #loading statment
    print("\nYour selection of", selection, "is loading...." )
    print('\n\n')
    #opens all of the sheets in the seasonal file
    kartData = sh.worksheet('Total Scores').get_all_values()
    RaceCount = sh.worksheet('Race Count').get_all_values()
    KartScore = sh.worksheet('Owned Score').get_all_values()
    Placement = sh.worksheet('Placement Stats').get_all_values()
    KVRStats = sh.worksheet('KVR Stats').get_all_values()
    #converts all to dataframs
    dfScores = pd.DataFrame(kartData[1:], columns=kartData[0])
    dfRaceCount = pd.DataFrame(RaceCount[1:], columns = RaceCount[0])
    dfKartScore = pd.DataFrame(KartScore[1:], columns = KartScore[0])
    dfPlacement = pd.DataFrame(Placement[1:], columns = Placement[0])
    dfKVR = pd.DataFrame(KVRStats[1:],columns = KVRStats[0])
    inputRace(dfScores,dfRaceCount, dfKartScore, dfPlacement, dfKVR,Track,Racers,Scores,TrackIndex)
 
 #edit a score
  elif selection == '2':
    print('Edit A Score is used to fix an error that may have occured while entering a race \nUse a positive number to increase a score, and a negative to decrease \nUse the net change you want to make as the value')
    Track = input("Name of Track: ")
    if Track in NickNameIndex:
      Track = NickNameIndex[Track]
    Racer = input("Names of Racer: ")
    Score = input("Score: ")
    print("\nYour selection of", selection, "is loading...." )
    print('\n\n')
    kartData = sh.worksheet('Total Scores').get_all_values()
    dfScores = pd.DataFrame(kartData[1:], columns=kartData[0])
    editAScore(dfScores, Track, Racer, Score,TrackIndex)

  #enter a winner
  elif selection == '3':
    Player = input("Winning Player: ")
    print("\nYour selection of", selection, "is loading...." )
    print('\n')
    Wins = sh.worksheet('GP Wins').get_all_values()
    dfWins = pd.DataFrame(Wins[1:], columns = Wins[0])
    enterWinner(dfWins, Player)
   
  #enter a dodge
  elif selection == '4':
    Player = input("Player Name: ")
    Count = input("How Many Dodges: ")
    print("\nYour selection of", selection, "is loading...." )
    print('\n')
    Shock = sh.worksheet('Shock Dodges').get_all_values()
    dfShock = pd.DataFrame(Shock[1:], columns = Shock[0])
    enterDodges(dfShock,Player,Count)

  #input blue shells
  elif selection == '5':
    Player = input("Player Name: ")
    hit = input("How Many times where they hit by a blue shell? ")
    dodge = input("How many did they dodge? ")
    print("\nYour selection of", selection, "is loading...." )
    print('\n')
    
    BlueShells = sh.worksheet("Blue Shells").get_all_values()
    dfBlueShell = pd.DataFrame(BlueShells[1:], columns = BlueShells[0])

    addBlueShells(dfBlueShell, Player, hit, dodge)

  elif selection == 'reset':
    answer = input("Are you sure you want to end the current season? ")
    
    if answer == 'yes' or answer == 'YES':
      if input("enter password: ") == 'resetseason123':
        #open up all time sheet
        change = 0
        all_time = gc.open_by_url('https://docs.google.com/spreadsheets/d/1nolol4P63e7WY-wRPnogcePkWuPPE3JF6QBRgXgF8gE/edit#gid=524408434')
        print("Ending Season....")
        #call end of season function
        newPoints,newRaceCount,newWins,newDodge,newOwned,newShells,newSeeds,oldPoints,oldRaceCount,oldWins,oldDodge,oldOwned,oldShells = end_season(sh,all_time,TrackIndex)
        #prompt to end the season
        save = input("are you sure you want to save? ")
        if save == 'YES' or save == 'yes':
          #if yes set change to = 1
          change = 1
          print('Finializing the data....')
        else:
          print('Canceling Change of Season...')
          print('End of Program')
      else: 
        print('password wrong')
    else:
      print('Stopping program')
    
 #invalid selection at start
  else: 
    print('invalid selection restart the program')



  ##COMMENT THIS OUT WHEN TESTING

  #upadates the google sheet, a psyudo save of the data
  TotalScores = sh.worksheet('Total Scores')
  RaceCount = sh.worksheet('Race Count')
  WinCount = sh.worksheet('GP Wins')
  Shock = sh.worksheet('Shock Dodges')
  KartScore = sh.worksheet('Owned Score')
  BlueShells = sh.worksheet('Blue Shells')
  Placement = sh.worksheet('Placement Stats')
  KVR = sh.worksheet('KVR Stats')
  
  #saves to the google sheet if nessassary, depending on the input of the user
  #saves if enter a race
  if selection =='1':
    set_with_dataframe(TotalScores, dfScores)
    set_with_dataframe(RaceCount,dfRaceCount)
    set_with_dataframe(KartScore,dfKartScore)
    set_with_dataframe(Placement,dfPlacement)
    set_with_dataframe(KVR,dfKVR)
    print("Success - The Database Has Been Updated")
  #saves if edit a score
  elif selection =='2':
    set_with_dataframe(TotalScores, dfScores)
    print("Success - The Database Has Been Updated")
 #saves when winner added
  elif selection =='3':
    set_with_dataframe(WinCount,dfWins)
    print("Success - The Database Has Been Updated")
  #saves when shock dodges are counted
  elif selection =='4':
    set_with_dataframe(Shock,dfShock)
    print("Success - The Database Has Been Updated")
  elif selection  == '5':
    #saves the data for blue shells
    set_with_dataframe(BlueShells,dfBlueShell)
    print("Success - The Database Has Been Updated")

  elif selection == 'reset':
    if change == 1:
      #opens up the all time list
      print("Saving All-Time Stats")
      aTotalScores = all_time.worksheet('Total Scores')
      aRaceCount = all_time.worksheet('Race Count')
      aWinCount = all_time.worksheet('GP Wins')
      aShock = all_time.worksheet('Shock Dodges')
      aKartScore = all_time.worksheet('Owned Score')
      aBlueShells = all_time.worksheet('Blue Shells')
      aSeeds = all_time.worksheet("All-Time Seeding")
      #saves the data for the all time list
      set_with_dataframe(aTotalScores,newPoints)
      set_with_dataframe(aRaceCount,newRaceCount)
      set_with_dataframe(aWinCount,newWins)
      set_with_dataframe(aShock,newDodge)
      set_with_dataframe(aKartScore,newOwned)
      set_with_dataframe(aBlueShells,newShells)
      set_with_dataframe(aSeeds,newSeeds)



      #sets all od the old ones to 0
      print("Resetting Season Stats...")
      set_with_dataframe(TotalScores, oldPoints)
      set_with_dataframe(RaceCount,oldRaceCount)
      set_with_dataframe(KartScore,oldOwned)
      set_with_dataframe(WinCount,oldWins)
      set_with_dataframe(Shock,oldDodge)
      set_with_dataframe(BlueShells,oldShells)

      print("The Seasonal Stats have been successfully merged into the All-Time Stats")
      print("Time for a new season to begin!")



# Main

In [134]:
#This is now the cell you need to run the program
#The old main cell was redefined from the main cell to the singular method below
#This is to reduce the amount of scrolling needed when inputing or viewing the data

######IMPORTS######
import pandas as pd
from google.colab import auth
import gspread
from google.auth import default
from gspread_dataframe import get_as_dataframe, set_with_dataframe
###################

#methodCall
RunKartniteStats()

Welcome to Kartnite Stats  v3.2.5 
Developed by: Patrick Marinich 

Below are the options, type the number of the option you would like
1. Input a Race 
2. Edit A Singular Score 
3. Enter A Grand Prix Winner 
4. Enter Shock Dodges 
5. Enter Blue Shell Stats
What Would You Like To Do: 5
Player Name: Karla
How Many times where they hit by a blue shell? 2
How many did they dodge? 0

Your selection of 5 is loading....


Karla has been hit by 2 Blue Shells
Karla has 0 Blue Shell Dodges
Success - The Database Has Been Updated
